# A Simple Reliability Example

https://pythonhosted.org/mcerp/basic_examples.html

adapted for example in

http://reliawiki.org/index.php/Reliability_Importance

In [1]:
# ! pip install mcerp3

Import required modules:

In [2]:
from mcerp3 import *  # N, U, Gamma, Beta, correlate, etc.
import mcerp3.umath as umath  # sin, exp, sqrt, etc.
import scipy.stats as ss  # for weibull distribution: exponweib
from scipy.special import gamma
import numpy as np

Analysis for three components configured reliability-wise in series following a Weibull distribution with β=3 and η1=1,000, η2=2,000 and η3=3,000.

In [3]:
t = 1000.0
loc = 0.0
beta = 3.0
eta1 = 1000.0
eta2 = 2000.0
eta3 = 3000.0

t1 = uv(ss.exponweib(1, beta, loc=loc, scale=eta1))
t2 = uv(ss.exponweib(1, beta, loc=loc, scale=eta2))
t3 = uv(ss.exponweib(1, beta, loc=loc, scale=eta3))

Check the mean (should be less than eta - the 63.2 percentile):

In [4]:
t1.mean

892.9782291086551

In [5]:
t2.mean

1785.945170600762

In [6]:
t3.mean

2678.94307974113

In [7]:
R1 = np.mean(t1 > t)
R1

0.3679

In [8]:
R2 = np.mean(t2 > t)
R2

0.8825000000000001

In [9]:
R3 = np.mean(t3 > t)
R3

0.9637

Compute the Reliability of the system using math for a components in a series (min):

In [10]:
ts = np.array([t1._mcpts, t2._mcpts, t3._mcpts]).min(axis=0)

In [11]:
Rs = np.mean(ts > t)
Rs

0.3137

## Check mean and reliability estimates vs. actual

In [12]:
def p_error(actual, sim):
    # percent error calculation
    result = round(100 * (sim - actual)/actual, 3)
    actual = round(actual, 4)
    sim = round(sim, 4)
    print('Actual = {}, Simulated = {}, Error = {}%'.format(actual, sim, result))

In [13]:
m1 = eta1*gamma(1 + 1/beta) # check mean 1
m2 = eta2*gamma(1 + 1/beta) # check mean 2
m3 = eta3*gamma(1 + 1/beta) # check mean 3
m1, m2, m3

(892.979511569249, 1785.959023138498, 2678.938534707747)

In [14]:
# Mean 1
p_error(m1, t1.mean)

Actual = 892.9795, Simulated = 892.9782, Error = -0.0%


In [15]:
# Mean 2
p_error(m2, t2.mean)

Actual = 1785.959, Simulated = 1785.9452, Error = -0.001%


In [16]:
# Mean 3
p_error(m3, t3.mean)

Actual = 2678.9385, Simulated = 2678.9431, Error = 0.0%


In [17]:
def weibull_surv(time, fr, slope):
    # Weibull distribution survival function
    # fr = failure rate (1/scale)
    # slope = shape parameter (slope of line on Weibull plot)
    return np.exp(-((time*fr)**slope))

In [18]:
r1 = weibull_surv(t, 1.0/eta1, beta)
r2 = weibull_surv(t, 1.0/eta2, beta)
r3 = weibull_surv(t, 1.0/eta3, beta)
rs = r1 * r2 * r3
np.round(np.array([r1, r2, r3, rs]), 4)

array([0.3679, 0.8825, 0.9636, 0.3128])

In [19]:
# Reliability 1
p_error(r1, R1)

Actual = 0.3679, Simulated = 0.3679, Error = 0.006%


In [20]:
# Reliability 2
p_error(r2, R2)

Actual = 0.8825, Simulated = 0.8825, Error = 0.0%


In [21]:
# Reliability 3
p_error(r3, R3)

Actual = 0.9636, Simulated = 0.9637, Error = 0.006%


In [22]:
# Reliability System
p_error(rs, Rs)

Actual = 0.3128, Simulated = 0.3137, Error = 0.272%
